In [1]:
# First cell - Setup paths
import sys
import os

# Setup paths as before
notebook_dir = os.path.dirname('__file__')
lerobot_root = os.path.abspath(os.path.join(notebook_dir, '../..'))
sys.path.append(lerobot_root)
os.chdir(lerobot_root)

In [2]:
import re
import cv2
import matplotlib.pyplot as plt
import time
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import json
import PIL
import google.generativeai as genai

In [3]:
from lerobot.common.policies.act.modeling_act import ACTPolicy
from lerobot.common.policies.pi0.modeling_pi0 import PI0Policy
from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig
from lerobot.common.robot_devices.robots.utils import make_robot_from_config
from lerobot.common.robot_devices.control_utils import busy_wait, log_control_info
from lerobot.common.robot_devices.robots.configs import So100RobotConfig

objc[6926]: Class AVFFrameReceiver is implemented in both /Users/shreyas/opt/anaconda3/envs/caferacer/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x3165c4798) and /Users/shreyas/opt/anaconda3/envs/caferacer/lib/libavdevice.59.7.100.dylib (0x3254a0778). One of the two will be used. Which one is undefined.
objc[6926]: Class AVFAudioReceiver is implemented in both /Users/shreyas/opt/anaconda3/envs/caferacer/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x3165c47e8) and /Users/shreyas/opt/anaconda3/envs/caferacer/lib/libavdevice.59.7.100.dylib (0x3254a07c8). One of the two will be used. Which one is undefined.


In [4]:
# Create camera config using proper config objects
cameras = {
    "laptop": OpenCVCameraConfig(
        camera_index=0,  # Built-in webcam
        fps=30,
        width=640,
        height=480
    ),
    "phone": OpenCVCameraConfig(
        camera_index=1,  # iPhone camera
        fps=30,
        width=640,
        height=480
    )
}

In [8]:
robot_cfg = So100RobotConfig(
            cameras=cameras,
            mock=False, 
            
        )
        
        # Create and connect robot
robot = make_robot_from_config(robot_cfg)
robot.connect()

Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


In [9]:
inference_time_s = 5
fps = 30
device = "mps"  # TODO: On Mac, use "mps" or "cpu"

ckpt_path = "/Users/shreyas/Downloads/checkpoints/100000/pretrained_model"
policy = ACTPolicy.from_pretrained(ckpt_path)
#policy = PI0Policy.from_pretrained("lerobot/pi0")
policy.to(device)

for _ in range(inference_time_s * fps):
    start_time = time.perf_counter()

    # Read the follower state and access the frames from the cameras
    observation = robot.capture_observation()
    
    # Convert to pytorch format: channel first and float32 in [0,1]
    # with batch dimension
    for name in observation:
        if "image" in name:
            observation[name] = observation[name].type(torch.float32) / 255
            observation[name] = observation[name].permute(2, 0, 1).contiguous()
        observation[name] = observation[name].unsqueeze(0)
        observation[name] = observation[name].to(device)

    # Compute the next action with the policy
    # based on the current observation
    action = policy.select_action(observation)
    # Remove batch dimension
    action = action.squeeze(0)
    # Move to cpu, if not already the case
    action = action.to("cpu")
    # Order the robot to move
    robot.send_action(action)
    
    dt_s = time.perf_counter() - start_time
    busy_wait(1 / fps - dt_s)

Loading weights from local directory


In [10]:
robot.disconnect()

In [ ]:
print(observation)

In [ ]:
print(action)